<a href="https://colab.research.google.com/github/alostmathematician/ISTA-366/blob/main/NB6_ISTA_322_S23_Simple_ETL_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wrapping up ETL with the L

The goal of this week's lesson is to teach you the process of uploading data to a database (You already followed a lesson to create a Postgres database on AWS). Now it's time to load data to it.

Before we do that though, we're going to extract data and transform it into a more useful analytical format. Thus, we'll be doing a full ETL pipeline today!

The data that we'll be extracting contains daily numbers of Covid-19 positive cases and deaths by each county and state in the US. Most dashboards or reporting metrics are going to want values 'rolled up' into a higher level. We're going to do this in two ways:

* We're going to want aggregate numbers by state. Hearing about high rates in some specific county might matter for some people, but most sources will want state-level numbers.

* We're also going to want to rollup our data into weekly aggregates vs. daily numbers. Daily numbers are problematic as some sources only report on specific days of the week. For example, hospitals might not update their stats on the weekends. Similarly U Arizona doesn't test on the weekends. This means that daily numbers aren't an accurate reflection of the overall trend, while weekly numbers are.

The last thing we'll do is some wrangling to make a unique primary key for each row.

Once that is done *then* we can upload the table to our database.

## Extracting our data

The data has been stored on my Google Drive.

We can bring the data in via our usual `pd.read_csv()`. Then we'll do a quick exploration to figure out what we need to do as far as transforms.

In [ ]:
# Get pandas
import pandas as pd

In [ ]:
# We'll call our data 'covid'
covid = pd.read_csv('https://docs.google.com/spreadsheets/d/1KhRW9lU_mm9svjy2T2u2Sx6UwTGcOfXx1jb1xJoxLpQ/gviz/tq?tqx=out:csv')

### Quick explore

Let's look at the head, a describe, and shape.

In [ ]:
# head
covid.tail()

In [ ]:
# describe
covid.describe()

In [ ]:
# dtypes
covid.dtypes

In [ ]:
# shape
covid.shape

OK, we can see a few things. We have six columns. Our shape tells us that we have over 600,000 rows. Our dtypes call indicates that our `date` column isn't currently a datetime. So if we're going to want to roll up by week then we'll need to convert that.

## Transforming our data

Time to get our transform going. We're going to do the following steps:

* Make `date` a datetime
* Aggregate our data by week and state and count up the number of cases/deaths
* Make a unique key that's needed for our database

### Make our datetime

You know how this is done. Just convert `date` in place.

In [ ]:
# Convert date to datetime
covid['date'] = pd.to_datetime(covid['date'])

In [ ]:
# Can we call a method on it to make sure it returns a datetime?
covid['date'].min()

### Groupby and aggregate

Time to groupby. We're going to group by two levels. The first is `state`, which is easy. The second is that we want to group by the calendar week. You could go and do something where you extract days and then have some rolling function that counts in groups of seven. But, pandas can actually group by calendar week if we use the `Grouper()` function!

To use `pd.Grouper()` you give it two arguments. The first is the key, which in this case is the `date` column. The second is the frequency that you want to group. We want to group on weeks so we can just specify `freq = 'w'`. The `W` tells it to group by calendar week. You can even specify what day you want the week to start on by adding a dash and a day abbreviation. So `W-MON` will tell it to group by weeks where weeks start on Monday.

For our `.agg()` we just want to sum up `daily_cases` and `daily_death` which will give us the weekly cases and deaths.

OK, let's do it and call the resulting object `covid_grouped`.

In [ ]:
# Test out using grouper. Play around with different freq abbreviations and observe the result
covid_grouped_test = covid.groupby([covid['state'], pd.Grouper(key = 'date', freq='...')]).agg({'daily_cases': ['sum'], 'daily_deaths': ['sum']})
covid_grouped_test.head(10)

In [ ]:
# Make covid_grouped for real
covid_grouped = covid.groupby([covid['state'], pd.Grouper(key = 'date', freq = 'W-MON')]).agg({'daily_cases' : ['sum'], 'daily_deaths': ['sum']})

In [ ]:
# Check the head of covid_grouped
covid_grouped.head()

Right now see how the state and date columns are in bold. That means they got assigned as indexes and not columns. Let's reset the index to bring them back as columns. We want them as columns for date manipulation and key generation later.

In [ ]:
# Reset index and check head
covid_grouped = covid_grouped.reset_index()
covid_grouped.head()

Let's also rename our columns. We'll leave the first two as 'state' and 'date', but change the last two to 'weekly_cases' and 'weekly_deaths'

In [ ]:
# rename to 'state', 'date', 'weekly_cases', 'weekly_deaths'
covid_grouped.columns = ['state', 'date', 'weekly_cases', 'weekly_deaths']
covid_grouped.head()

### A quick validation of our data

Let's take a minute and make sure our data makes sense. We'll extract out data from just Arizona and then do a quick barplot of the number of weekly cases in AZ.

In [ ]:
# Extract just AZ observations
az_data = covid_grouped[covid_grouped['state'] == 'Arizona']

In [ ]:
#For local machine you may need to install seaborn
!pip install seaborn

In [ ]:
# Make barplot using seaborn
# The x-axis is a mess but data look right based on what we know about Covid in AZ
import seaborn as sns
sns.barplot(x = 'date', y = 'weekly_cases', data = az_data)

## Transforms - Making a key

OK, our `covid_grouped` dataframe is now rolled up into a format that we want. But since we're going to push to a database we need to also make a key. A **primary key** is a unique identifying value for a row of the table. Such a key must not be duplicated in any of the rows (it wouldn't be unique if it was!).

So, we need to make a key. What should we use? There isn't a firm rule here, but ideally you want something that is easy to generate. We could imagine making a table of a bunch of random values and selecting from that. The only issue is that if we want to add more data, every time you do you'd need to make sure the key you're giving a row hasn't already been used. You could also imagine making the ID just from a squence, but again if you update you need to know where the sequence ends.

Instead of either of those two options we can do something simpler. Let's just take the date in YYYYMMDD format and then add on a code for the state. Any updates obviously won't duplicate dates. Adding on the state code ensures that each date is then unique as no states will have the same state code.

To make the key we'll do the following steps:
* Make a column called `week_start` that takes the date the grouped week began on and converts it back into a string.
* We'll remove the dashes from the current YYYY-MM-DD format
* Make numeric values for each state in the dataset
* Add those state numeric values back to the date string

### Making our date string

First we'll use our basic wrangling skills to take our date column and convert it back to a string with `.astype(str)`. We'll call the new column 'week_start'.

In [ ]:
# Convert to string and check
covid_grouped['week_start'] = covid_grouped['date'].astype(str)
covid_grouped.dtypes # can see that week_start is an object

Removing symbols from strings is really common. Luckily it's also quite easy here and we don't need regular expression. `.replace()` allows you to just specify what symbol you want it to remove first, in this case the "-", and then what you want to replace it with, in this case nothing.

Let's do this operation on 'week_start'.

In [ ]:
covid_grouped.head()

In [ ]:
# Remove dash from week start
covid_grouped['week_start'] = covid_grouped['week_start'].str.replace('-', '')
covid_grouped.head()

### Making our state names into numeric factors

We want to assign each unique level in the 'state' column it's own numeric ID value. Pandas has a function called `factorize()` that does just that. As an argument you feed it a column that has multiple unique characters, and it'll return a unique numeric value for each. Here's an example on some test data

```
>>> states = ['washington', 'washington', 'arizona', 'california']
>>> pd.factorize(states)
(array([0, 0, 1, 2]),
 array(['washington', 'arizona', 'california'], dtype=object))
```
You can see that `pd.factorize()` took 'states` and gave the unique values a corresponding numeric value starting with zero. Test it out below if you want.


In [ ]:
# If you want to test it out
states = ['washington', 'washington', 'arizona', 'california']
pd.factorize(states)

Let's apply factorize to create a new column called 'state_id'. Note how above `factorize()` returned two arrays. The first had the ID, the second the original level. We'll need to call just that first array with `[0]` as that's all we want in the new column.

I'm also going to convert the numeric straight to a string using `.astype(str)` again. This will save us a step when we add the `state_id` string to the `week_start` string.

In [ ]:
covid_grouped['state_id'] = pd.factorize(covid_grouped['state'])[0].astype(str)
covid_grouped # check it

Great. We can see we have our 'state_id' column now. Let's join that to the front of 'week_start' so this way our key format is IDYYYYMMDD where ID at the start of the string is the state ID.

If you want to merge two strings in different columns you can literally just add them together. So let's add the 'state_id' column to the 'week_start' column. Let's create a new column called 'wk_st_id' for "week state id."

In [ ]:
# Add strings
covid_grouped['wk_st_id'] = covid_grouped['state_id'] + covid_grouped['week_start']
covid_grouped #check

## Making our table of state keys

Now that we have our encoded key for our covid_grouped table we need to do two final things. First, let's make a second table that contains the relationship between the state_id and the state. This way people can make sql queries on the covid data using the state name if they wanted.

After that we'll drop the unnecessary columns from `covid_grouped`.

Making our 'states' table is easy. Just select the two columns of 'state_id' and 'state' and use `drop_duplicates() to get just unique values. This way we'll have each state only once. Call the resulting dataframe 'states'

In [ ]:
# Make states by selecting columns and dropping duplicates
states = covid_grouped[['state_id', 'state']].drop_duplicates()
states.head() # check

Now we'll drop 'week_start' and 'state' from `covid_grouped` as those columns are redundant. The week start is just the 'date' column which we'll keep. And we have a 'state_id' column so we don't need the state name.

`.drop()` is useful here.

In [ ]:
# Drop columns
covid_grouped = covid_grouped.drop(columns=['week_start', 'state'])
covid_grouped.head()

## Pushing to our/your SQL database

Alright, let's go and push our two tables to the database that we set up earlier in the week.

Pushing data to a database has two steps.

1. We define our table columns and the datatypes within in
2. We upload data into the columns that we created
3. We commit the changes

Steps 1 and 2 are done using SQL syntax in a similar structure as we used to make queries. We'll write our commands and then execute them using a cursor just like before. Then we'll commit our changes and close our cursor.

### Import SQL functions

First, go and run the functions we've been using. **Update** the `get_conn_cur()` function so that it connects to your database!

In [ ]:
#If you are using AWS, use this cell
import psycopg2
def get_conn_cur():
 # UPDATE WITH YOUR INFO!

 conn = psycopg2.connect(
    host="YOUR DB HOST INFO HERE",
    database="YOUR DB NAME HERE",
    user="YOUR DB USER HERE",
    password="YOUR DB PASSWORD HERE",
    port='YOUR DB PORT HERE'
    )

 cur = conn.cursor()
 return(conn, cur)

In [ ]:
#If you using local MySQL use this cell. (Note: if you are using local MySQL you must run this notebook on your local Jupyter, it won't work on Google Colab)
import mysql.connector
def get_conn_cur():
    cnx = mysql.connector.connect(user='root', password='',
          host='127.0.0.1',
          database='YOUR DB NAME HERE');
    return (cnx, cnx.cursor())

In [ ]:
# Run or old functions in case we want to check our data!

# run_query function
def run_query(query_string):

 conn, cur = get_conn_cur() # get connection and cursor

 cur.execute(query_string) # executing string as before

 my_data = cur.fetchall() # fetch query data as before

 # here we're extracting the 0th element for each item in cur.description
 colnames = [desc[0] for desc in cur.description]

 cur.close() # close
 conn.close() # close

 return(colnames, my_data) # return column names AND data

# Column name function for checking out what's in a table
def get_column_names(table_name): # arguement of table_name
 conn, cur = get_conn_cur() # get connection and cursor

 # Now select column names while inserting the table name into the WERE
 column_name_query = """SELECT column_name FROM information_schema.columns
    WHERE table_name = '%s' """ %table_name

 cur.execute(column_name_query) # exectue
 my_data = cur.fetchall() # store

 cur.close() # close
 conn.close() # close

 return(my_data) # return

# Check table_names
def get_table_names():
  conn, cur = get_conn_cur() # get connection and cursor

  # query to get table names
  table_name_query = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public' """

  cur.execute(table_name_query) # execute
  my_data = cur.fetchall() # fetch results

  cur.close() #close cursor
  conn.close() # close connection

  return(my_data) # return your fetched results

# make sql_head function
def sql_head(table_name):
 conn, cur = get_conn_cur() # get connection and cursor

 # Now select column names while inserting the table name into the WERE
 head_query = """SELECT * FROM %s LIMIT 5; """ %table_name

 cur.execute(head_query) # exectue
 colnames = [desc[0] for desc in cur.description] # get column names
 my_data = cur.fetchall() # store first five rows

 cur.close() # close
 conn.close() # close

 df = pd.DataFrame(data = my_data, columns = colnames) # make into df

 return(df) # return

# drop a table from your rdb (if you try to create a table that already exists, it'll throw an error)
def my_drop_table(tab_name):
  conn, cur = get_conn_cur()
  tq = """DROP TABLE IF EXISTS %s CASCADE;""" %tab_name
  cur.execute(tq)
  conn.commit()

## Create weekly table in database

Let's go and make a table called 'weekly' that we'll insert our main 'covid_grouped' dataset into'

The main call to make a table is `CREATE TABLE new_table_name ()`. Inside the parentheses you then give it your column name, your datatype, and if you can have null values in there. We want to make five columns. The first one we want to be our 'wk_st_id' which we'll set as an integer and a primary key.

I've written the query for you.

**NOTE** - `VARCHAR(255)` is the way to specify a character column. Inside the parentheses is the max length allowed. 'state_id' doesn't need to be a character, but I specified it as one to demonstrate some variety. It's also how we already had the datatype specified in our 'covid_grouped' dataframe. We could specify the column as an integer but then we'd have to also change 'state_id' in 'covid_grouped' to an integer so the datatypes being pushed match.

In [ ]:
# drop the table if it already exists (uncomment)
# my_drop_table('weekly_stats')

# query with wk_st_id as an integer and primary key
# date as a timestamp
# cases and deaths as intergers
# state_id as characters
tq = """CREATE TABLE weekly_stats (
     wk_st_id INT PRIMARY KEY,
     date timestamp NOT NULL ,
     weekly_cases BIGINT NOT NULL,
     weekly_deaths BIGINT NOT NULL,
     state_id BIGINT NOT NULL
     );"""

Now that we have that let's get a connection and cursor object

In [ ]:
# Get our connection and cursor and assign to conn and cur
conn, cur = get_conn_cur()


To create the table we can just execute the query using `cur.execute()`.

In [ ]:
# execute our query we wrote above
cur.execute(tq)

And finally just commit the table. Doing so just finalizes everything you did and will make it accessable to other users.

In [ ]:
# commit
conn.commit()

Let's check that our table name and column names are in there using the functions we made in previous lessons.

In [ ]:
# Is the table there?
get_table_names()

In [ ]:
# All of our column names?
get_column_names(table_name='weekly_stats')

### Updating our table

I actually made a mistake above. I said the primary key should be an integer, but actually it needs to be something called a big interger. This is because regular integers can only store values between -2147483648 and +2147483647. Our current wk_st_id values get larger than that. So, we need to change the datatype of that column!

To do this, or change any datatype for that matter, we write another query. In this case, the main arguments are ALTER TABLE and ALTER COLUMN. The first just points to the table in the database, the second the column in that database. Then you just tell it what datatype you want to use. the general format is as follows:

```
ALTER TABLE table_name
 ALTER COLUMN column_to_update TYPE datatype_to_use
```

I've written the query below. We'll create a cursor and commit everything in one go just like above.

In [ ]:
conn, cur = get_conn_cur()
cur = conn.cursor()
uq = """ALTER TABLE weekly_stats
     ALTER COLUMN wk_st_id TYPE BIGINT;"""
cur.execute(uq)
conn.commit()

We can run a quick query to get the datatypes. Remember how we called the 'information_schema' before to get column names? We can also get the data types by selecting that information.

In [ ]:
conn, cur = get_conn_cur()
cur = conn.cursor()
uq = """SELECT column_name, data_type
     FROM information_schema.columns
     WHERE table_name = 'weekly_stats';"""
cur.execute(uq)
print(cur.fetchall())
conn.close()

Great, we can see all of our column names and corresponding datatypes. We can see that 'wk_st_id' is now a 'bigint.' This brings up an important point when designing a database. You need to be sure to make your columns are big enough to store approved values now, but also in the future.

## Make states table

Can you go and make a second table called 'states'? We're going to want to take that 'states' dataframe and add it to our database. This one should have only two columns, both with character datatypes like above. The primary key should be 'state_id'

In [ ]:
states.head()

In [ ]:
# Make your states table
conn, cur = get_conn_cur()
cur = conn.cursor()
tq = """CREATE TABLE ... (
     ...,
     ...
     );"""
cur.execute(tq)
conn.commit()
conn.close()

In [ ]:
# check that the table is tere
get_table_names()

In [ ]:
# Check that the column names are there
get_column_names(table_name='states')
sql_head('states')

In [ ]:
# Might as well check the datatypes
conn, cur = get_conn_cur()
cur = conn.cursor()
uq = """SELECT column_name, data_type
     FROM information_schema.columns
     WHERE table_name = 'states';"""
cur.execute(uq)
print(cur.fetchall())
conn.close()

## Putting information in the database

Now we have a couple empty tables sitting in our database waiting for data. As you can imagine, the general process is an extension of what we did above. We write a query that specifies the table and columns we want to insert data into, and then what data we want to put in there.

The main argument to do this is INSERT into followed by VALUES. The generalized string format is:

```
"INSERT INTO table_name(column_names) VALUES (values_for_each_column)"
```

This process is typically done row-wise meaning you insert one row at a time. For example, this string would add just a single row of data into our 'weekly_stats' table:

```
"INSERT INTO weekly_stats(wk_st_id,date,weekly_cases,weekly_deaths,state_id
 VALUES(0020200316, 2020-03-16 00:00:00, 19, 0, 00)"
```

**OK, but does this mean we have to write and execute a query for each and every row in our dataframe?** No, of course not. While you could update it a single row at a time using `cur.execute()` and loop through every row of your dataset, that would be super slow as you'd be generating a cursor, executing, and closing each and every time.

Instead you can use `cur.executemany()`. This allows you to, well, execute many actions at once. It is a bit more involved in the string you'll have to write as you'll have to use string formatting (`%s`) to generate the string for each row. But, this is simpler and MUCH MUCH faster than using a loop with single operations.

There are some other bits to make this work, but we'll cover them as we go. Let's get started!

First, we need to reorder our columns in the dataframe to match the column order of the database.

In [ ]:
# What's the order of our weekly_stats column?
get_column_names(table_name='weekly_stats')

In [ ]:
# Reorder covid_grouped to match
covid_grouped = covid_grouped[['wk_st_id', 'date', 'weekly_cases', 'weekly_deaths', 'state_id']]
covid_grouped.head()

OK, first think to point out. SQL wants to receive the data as a tuple. Each row of ours is currently a row in a dataframe. So we need to convert each row to a tuple. We can use numpy to do this. Below I write a list comprehension to do this. List comprehensions are essentially simplified for loops.

The first part is the action. It's saying 'make a tuple with x'. The second part is just saying the variable name in the sequence. The third part is the list of data. In this case we're making a list of data by converting our dataframe to a numpy array. Play around making a cell and running `covid_grouped.to_numpy()` if you want to see more specifically what this is doing. I'm going to just run the list comprehension for now.

In [ ]:
import numpy as np

covid_np = covid_grouped.to_numpy();
#Mysql does not understand the timestamp from python so we can convert the timestamp to string.
covid_np[:,1] = np.vectorize(lambda x: str(x))(covid_np[:,1])
data_tups = [tuple(x) for x in covid_np]


What's in `data_tups`? Let's look at the 3rd position.

In [ ]:
# Check out an example
data_tups[2]

Great, so now that's in a format that's ready for VALUES part of our INSERT string. Now we can write our insert string that will take those values.

Remember from before that %s allows you to insert a value into a string. So if we did something like `'I need %s points to pass' % points_needed` it would insert whatever 'points_needed' is defined as into that string.

In [ ]:
# and example
points_needed = 20
'I need %s points to pass' % points_needed

What we need to do for our query is insert one of our tuples from 'data_tups' into VALUES. WE have five values in each tuple, so we need to just write '%s inside that string five times. Let's write our insert string.

In [ ]:
iq = """INSERT INTO weekly_stats(wk_st_id,date,weekly_cases,weekly_deaths,state_id) VALUES(%s, %s, %s, %s, %s);"""
iq

You can test if it's working by trying to add a tuple to your insert string.

In [ ]:
# Does it work?
iq % data_tups[4]

### Finally pushing data to the database

Now we can finally use `executemany() to push our database. This'll take two arguments. The first is the query string, the section is our tuples. It'll go and make an insert string for each and every tuple and therefore insert each row. We still need to open a connection, make a cursor, close, and commit.

**NOTE** this will take a minute or so!

In [ ]:
# Upload data
conn, cur = get_conn_cur()
cur.executemany(iq, data_tups)
conn.commit()
conn.close()

Let's give it a check with a simple query

In [ ]:
sq = """ SELECT COUNT(DISTINCT(wk_st_id)) FROM weekly_stats
     LIMIT 5;"""
run_query(sq)

**SUCCESS**

### Add data to states table

Wonderful! Can you now go and populate the 'states' table?

You'll need to make tuples and modify the insert statement to take only two arguments along with the proper table name and column names

In [ ]:
import numpy as np
...

In [ ]:
get_column_names(table_name='states')

In [ ]:
# new insert query
iq = """...;"""

In [ ]:
# Upload data
...

## Run some test queries!

In [ ]:
# Test query to look at the first five rows of states
sq = """ SELECT * FROM states
     LIMIT 5;"""
run_query(sq)

In [ ]:
# And weekly_stats
sq = """ SELECT * FROM weekly_stats
     LIMIT 5;"""
run_query(sq)

In [ ]:

# Do a subquery getting just weekly stats where the state is 'Arizona'
sq = """ SELECT * FROM weekly_stats
     WHERE state_id = (SELECT state_id FROM states WHERE state = 'Arizona')
     LIMIT 5;"""
run_query(sq)